In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import time

In [2]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)

        output = F.log_softmax(x, dim=1)

        return output

In [3]:
# Define normalization
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

# Load dataset
dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../data", train=False, transform=transform)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=64, shuffle=True)

In [5]:
# Build the model we defined above
model = Lenet()

# Add weight_decay parameter to optimizer for L2 regularization
optimizer = optim.Adadelta(model.parameters(), lr=1, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

model.train()

start_time = time.time()

for epoch in range(1, 6):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

    scheduler.step()

train_time = time.time() - start_time
print(f'training time = {train_time} seconds')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306099
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.257048
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.015944
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.555815
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.087375
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.229237
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.703261
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.634183
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.769441
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.809484
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.557343
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.551531
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.428897
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.412151
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.692472
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.334432
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.347286
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.370552
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.389605
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.459547
Train Epoch: 1 [12800/60000 (

In [6]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction="sum"
        ).item()  # sum up batch loss

        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability

        correct += pred.eq(target.view_as(pred)).sum().item()


test_loss /= len(test_loader.dataset)

print(
    "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset),
    )
)


Test set: Average loss: 0.0718, Accuracy: 9775/10000 (98%)



In [12]:
torch.save(model.state_dict(), "mnist_cnn.pt")